<a href="https://colab.research.google.com/github/dustin-kang/Proj1_GameDesignAnalysis/blob/main/code/Proj1_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. EDA**

## 1-1. 데이터 확인

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [144]:
data = pd.read_csv('/content/vgames2.csv')
data = data.drop(columns="Unnamed: 0")
data.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [145]:
data.shape

(16598, 9)

16598개의 데이터가 존재하고 9개의 컬럼이 있음을 확인할 수 있었다.

In [146]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16327 non-null  float64
 3   Genre        16548 non-null  object 
 4   Publisher    16540 non-null  object 
 5   NA_Sales     16598 non-null  object 
 6   EU_Sales     16598 non-null  object 
 7   JP_Sales     16598 non-null  object 
 8   Other_Sales  16598 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.1+ MB


In [147]:
data[data.Year.isna() == True].head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
31,Rock Band,X360,NaN,Misc,Electronic Arts,1.93,0.34,0,0.21
109,Yu-Gi-Oh! 5D's Wheelie Breakers (JP sales),Wii,NaN,Racing,Unknown,0,0,0.02,0
273,Jewel Link Chronicles: Mountains of Madness,DS,NaN,Puzzle,Avanquest,0,0.06,0,0.01
358,Majesty 2: The Fantasy Kingdom Sim,X360,NaN,Simulation,Unknown,0.03,0,0,0
430,Famista 64,N64,NaN,Sports,Namco Bandai Games,0,0,0.17,0.03


In [148]:
data[data.Genre.isna() == True].head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
50,Ape Escape: On the Loose,PSP,2005.0,NaN,Sony Computer Entertainment,0.52,0.01,0.12,0.05
408,NBA Ballers,XB,2004.0,NaN,Midway Games,0.37,0.11,0,0.02
1328,NFL Street,XB,2004.0,NaN,Electronic Arts,0.49,0.15,0,0.02
1582,SingStar Singalong With Disney,PS2,2008.0,NaN,Sony Computer Entertainment,0,0.04,0,0.57
1971,Pokemon X/Pokemon Y,3DS,2013.0,NaN,Nintendo,5170K,4.05,4.34,0.79


In [149]:
data[data.Publisher.isna() == True].head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
272,UK Truck Simulator,PC,2010.0,Simulation,NaN,0,0.03,0,0
1313,All Grown Up!: Game Boy Advance Video Volume 1,GBA,2004.0,Misc,NaN,0.17,0.06,0,0
1560,Teenage Mutant Ninja Turtles: Game Boy Advance...,GBA,2004.0,Misc,NaN,0.12,0.04,0,0
1789,Nicktoons Collection: Game Boy Advance Video V...,GBA,2004.0,Misc,NaN,0.16,0.06,0,0
1879,Monster Hunter Frontier Online,PS3,NaN,Role-Playing,NaN,0,0,0.05,0


In [150]:
data.isna().sum()

Name             0
Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

Data의 결측치를 확인해봤을 경우,
- `Year` : 271개
- `Genre` : 50개
- `Publisher` : 58개

등의 데이터에서 결측치가 있는 것으로 확인이 되었다.

추가적으로, Sales(출고량) 데이터는 Object 형이므로 이상치가 있음이 확인된다.


## 1-2. 데이터 이상치 및 결측치 보완


### 1-2-1. `Year` 데이터 이상치 보완
만약에 Year 데이터에도 잘못 기재된 이상치가 존재할 수 있다. \
그래서 이상치가 있는지 확인을 해보고 수정하기로 했다.

In [151]:
years = data.Year.unique()

tmp = []
for i in years:
  if i < 1900:
    tmp.append(i)

tmp

[9.0,
 97.0,
 11.0,
 15.0,
 3.0,
 96.0,
 6.0,
 8.0,
 1.0,
 5.0,
 4.0,
 10.0,
 98.0,
 7.0,
 16.0,
 86.0,
 14.0,
 95.0,
 2.0,
 13.0,
 0.0,
 12.0,
 94.0]

역시나 이상수치가 있다는 것을 알게되었다.
위 이상수치를 `tmp`에 담았으니,
- 80년대와 90년대를 나타내는 것은 1900를 더하고
- 그외 숫자들은 2000년을 더해

이상수치를 보완하는것이 좋겠다.

In [152]:
condition1 = data.Year.values <= 80
condition2 = data.Year.values >= 80
condition3 = data.Year.values <= 100

data.loc[condition1, 'Year'] += 2000
data.loc[(condition2) & (condition3), 'Year'] += 1900

data.Year.value_counts()

2009.0    1431
2008.0    1428
2010.0    1259
2007.0    1202
2011.0    1139
2006.0    1008
2005.0     941
2002.0     829
2003.0     775
2004.0     763
2012.0     657
2015.0     614
2014.0     582
2013.0     546
2001.0     482
1998.0     379
2000.0     349
2016.0     344
1999.0     338
1997.0     289
1996.0     263
1995.0     219
1994.0     121
1993.0      60
1981.0      46
1992.0      43
1991.0      41
1982.0      36
1986.0      21
1989.0      17
1983.0      17
1990.0      16
1987.0      16
1988.0      15
1985.0      14
1984.0      14
1980.0       9
2017.0       3
2020.0       1
Name: Year, dtype: int64

### 1-2-2. `Year`, `publisher`, `genre` 결측치 보완
이상치에 대해서 보완이 끝났으니 이젠 271개의 Year 데이터의 결측치를 보완하기로 했다.

데이터 수 16598 -> 16241로 제거 했어도 데이터 크기에 크게 영향을 미치지 않은 것 같다.

>물론 selenium이나 크롤링으로 데이터를 수집할 수 있으나, 아직 이 크롤링 기법들은 섹션에서 배우지 않았음을 가정하에 진행 하였다.

In [153]:
data[data.Year.isna()]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
31,Rock Band,X360,NaN,Misc,Electronic Arts,1.93,0.34,0,0.21
109,Yu-Gi-Oh! 5D's Wheelie Breakers (JP sales),Wii,NaN,Racing,Unknown,0,0,0.02,0
273,Jewel Link Chronicles: Mountains of Madness,DS,NaN,Puzzle,Avanquest,0,0.06,0,0.01
358,Majesty 2: The Fantasy Kingdom Sim,X360,NaN,Simulation,Unknown,0.03,0,0,0
430,Famista 64,N64,NaN,Sports,Namco Bandai Games,0,0,0.17,0.03
...,...,...,...,...,...,...,...,...,...
16240,Freaky Flyers,GC,NaN,Racing,Unknown,0.01,0,0,0
16275,Space Invaders,2600,NaN,Shooter,Atari,2.36,0.14,0M,0.03
16278,Madden NFL 11,Wii,NaN,Sports,Unknown,0.7,0,0,50K
16425,Yoostar on MTV,X360,NaN,Misc,Unknown,0.06,0,0,0


In [154]:
data.Year.isna().sum()

271

In [155]:
data_1 = data.copy() #  Publisher 은  반드시 들어갈 내용이 없기 때문

In [156]:
data = data.dropna()

In [157]:
data.isna().sum()

Name           0
Platform       0
Year           0
Genre          0
Publisher      0
NA_Sales       0
EU_Sales       0
JP_Sales       0
Other_Sales    0
dtype: int64

In [112]:
data

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16593,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16594,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16595,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16596,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


In [113]:
data.Genre.unique()

array(['Action', 'Adventure', 'Misc', 'Platform', 'Sports', 'Simulation',
       'Racing', 'Role-Playing', 'Puzzle', 'Strategy', 'Fighting',
       'Shooter'], dtype=object)

이제 마지막으로 `Year` 데이터를 int 형식으로 보기 편하게 바꿔주려고 한다.

In [114]:
data[data.columns[2]] = data[data.columns[2]].astype(int)

### 1-2-3. `Sales` 데이터 이상치 보완

처음 봤을 때, 단위의 수를 이해할 수 없었다.
 하지만 예상이 가는건,

In [115]:
data.NA_Sales.value_counts()

0        4367
0.02      524
0.03      518
0.01      516
0.04      515
         ... 
80K         1
11.27       1
6.05        1
2.66M       1
0.29M       1
Name: NA_Sales, Length: 489, dtype: int64

In [116]:
data.EU_Sales.value_counts()

0        5543
0.01     1443
0.02     1230
0.03      905
0.04      719
         ... 
2.86        1
2.12        1
1350K       1
8.03        1
4.52        1
Name: EU_Sales, Length: 360, dtype: int64

몇몇 작은 데이터들이 `M` 이나 `K` 같은 키워드를 쓰는 것으로 확인할 수 있다.

 정확한 파악을 위해 M, K가 포함된(`contain`)된 데이터들을 따로 데이터프레임으로 만들었습니다.

In [117]:
data[data.NA_Sales.str.contains('K')]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
10,Ford Racing,PS,2001,Racing,Empire Interactive,480K,0.33M,0K,0.06
45,Resident Evil: Revelations 2,PS3,2015,Action,Capcom,60K,0.08,0.16,0.03
145,TOCA Race Driver 3,PC,2006,Racing,Codemasters,0K,0.02,0,0
878,Tears to Tiara Anecdotes: The Secret of Avalon,PS3,2009,Strategy,Aqua Plus,0K,0,0.04,0
1086,Pro Evolution Soccer 2014,3DS,2013,Action,Konami Digital Entertainment,0K,0,0.19,0
...,...,...,...,...,...,...,...,...,...
15647,Championship Manager 2010,PC,2009,Sports,Eidos Interactive,0K,0.01,0,0
16194,Vin Diesel: Wheelman,PS3,2009,Racing,Ubisoft,120K,0.15,0,0.06
16330,Midway Arcade Treasures,PS2,2003,Misc,Midway Games,720K,0.56,0,0.19
16492,Mortal Kombat,PSV,2012,Fighting,Warner Bros. Interactive Entertainment,470K,0.18,0,0.12


In [118]:
data[data.NA_Sales.str.contains('M')]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
447,Medal of Honor: Warfighter,X360,2012,Action,Electronic Arts,0.58M,0.47,0.01,0.1
458,F1 2010,PS3,2010,Racing,Codemasters,0.25M,0.76,0.08,0.27
577,Mario Kart 64,N64,1996,Racing,Nintendo,5.55M,1.94,2.23,0.15
625,The Hobbit,GBA,2003,Platform,Vivendi Games,0.14M,0.05,0,0
672,Star Wars The Clone Wars: Lightsaber Duels,Wii,2008,Action,LucasArts,1.23M,0.3,0,0.14
...,...,...,...,...,...,...,...,...,...
15986,Juiced 2: Hot Import Nights,PC,2007,Racing,THQ,0M,0.01,0,0
16152,Taiko no Tatsujin Wii,Wii,2008,Misc,Namco Bandai Games,0M,0,0.67,0
16169,Medal of Honor: Airborne,X360,2007,Shooter,Electronic Arts,0.45M,0.53,0.01,0.12
16408,NASCAR 06: Total Team Control,PS2,2005,Racing,Electronic Arts,0.53M,0.02,0,0.09


확인해보니, 기준 단위는 백만단위 인것으로 알 수 있었다.

- M : 백만을 의미하며 그냥 'M'이라는 글자만 지워도 괜찮다고 생각한다. (1,000,000 = 1M)
- K : 숫자 1,000을 의미한다. 그래서 10,000을 10K로 표기하곤 한다.

> 60K = 60,000 = 0.06  

즉, K는 1000을 나눠 주면 된다.


In [119]:
# Sales
Sales = data.iloc[:,5:]

def to_M(data):
  for col in Sales:
    # K, M가 있는 데이터 가져오기
    sale_k = data.loc[data[col].str.contains('K'), col]
    sale_m = data.loc[data[col].str.contains('M'), col]
    # `replace` 를 이용하여 'K'나 'M'을 문자열 수정하기
    sale_k = sale_k.str.replace('K','').astype(float) / 1000
    sale_m = sale_m.str.replace('M','').astype(float) 
    # 기존 type이 string이기 때문에 우선 string으로 변경해서 저장
    data.loc[data[col].str.contains('K'), col] = sale_k.astype(str)
    data.loc[data[col].str.contains('M'), col] = sale_m.astype(str)

to_M(data)
data.NA_Sales.value_counts()

0       4367
0.02     536
0.03     523
0.01     521
0.05     519
        ... 
5.54       1
1.62       1
4.09       1
4.76       1
4.1        1
Name: NA_Sales, Length: 408, dtype: int64

In [120]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         16241 non-null  object
 1   Platform     16241 non-null  object
 2   Year         16241 non-null  int64 
 3   Genre        16241 non-null  object
 4   Publisher    16241 non-null  object
 5   NA_Sales     16241 non-null  object
 6   EU_Sales     16241 non-null  object
 7   JP_Sales     16241 non-null  object
 8   Other_Sales  16241 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.2+ MB


In [121]:
col = data.columns[5:]
data[col] = data[col].astype('float')

In [122]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16241 non-null  object 
 1   Platform     16241 non-null  object 
 2   Year         16241 non-null  int64  
 3   Genre        16241 non-null  object 
 4   Publisher    16241 non-null  object 
 5   NA_Sales     16241 non-null  float64
 6   EU_Sales     16241 non-null  float64
 7   JP_Sales     16241 non-null  float64
 8   Other_Sales  16241 non-null  float64
dtypes: float64(4), int64(1), object(4)
memory usage: 1.2+ MB


깔끔하게 정리된 것을 확인할 수 있었다.

### 1-2-4. 새로운 컬럼 추가 (전체 출고량)

In [123]:
data = data.reset_index().drop(columns='index')

In [124]:
Total_sales = []

for i in data.index:
  Total_s = data.iloc[i, 5:10].sum()
  Total_sales.append(round(Total_s,2))

data['Total_Sales'] = Total_sales


In [125]:
data

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
0,Candace Kane's Candy Factory,DS,2008,Action,Destineer,0.04,0.00,0.00,0.00,0.04
1,The Munchables,Wii,2009,Action,Namco Bandai Games,0.17,0.00,0.00,0.01,0.18
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010,Adventure,Alchemist,0.00,0.00,0.02,0.00,0.02
3,Deal or No Deal: Special Edition,DS,2010,Misc,Zoo Games,0.04,0.00,0.00,0.00,0.04
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010,Platform,D3Publisher,0.12,0.09,0.00,0.04,0.25
...,...,...,...,...,...,...,...,...,...,...
16236,Ice Age 2: The Meltdown,GC,2006,Platform,Vivendi Games,0.15,0.04,0.00,0.01,0.20
16237,Rainbow Islands: Revolution,PSP,2005,Action,Rising Star Games,0.01,0.00,0.00,0.00,0.01
16238,NBA 2K16,PS3,2015,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13,0.79
16239,Toukiden: The Age of Demons,PSV,2013,Action,Tecmo Koei,0.05,0.05,0.25,0.03,0.38


In [126]:
data.to_csv("vgame_eda.csv", index=False)

### 번외) `Publisher`의 na를 Various로 변경했을 때


In [158]:
data_1.loc[data_1.Publisher.isna(), "Publisher"] = "Unknown"

In [159]:
data_1.isna().sum()

Name             0
Platform         0
Year           271
Genre           50
Publisher        0
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

In [160]:
data_1 = data_1.dropna()

In [161]:
data_1['Year'] = data_1.iloc[:,2].astype(int)

In [162]:
data_1

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16593,Ice Age 2: The Meltdown,GC,2006,Platform,Vivendi Games,0.15,0.04,0,0.01
16594,Rainbow Islands: Revolution,PSP,2005,Action,Rising Star Games,0.01,0,0,0
16595,NBA 2K16,PS3,2015,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16596,Toukiden: The Age of Demons,PSV,2013,Action,Tecmo Koei,0.05,0.05,0.25,0.03


In [165]:
# Sales
Sales = data_1.iloc[:,5:]

def to_M(data):
  for col in Sales:
    # K, M가 있는 데이터 가져오기
    sale_k = data.loc[data[col].str.contains('K'), col]
    sale_m = data.loc[data[col].str.contains('M'), col]
    # `replace` 를 이용하여 'K'나 'M'을 문자열 수정하기
    sale_k = sale_k.str.replace('K','').astype(float) / 1000
    sale_m = sale_m.str.replace('M','').astype(float) 
    # 기존 type이 string이기 때문에 우선 string으로 변경해서 저장
    data.loc[data[col].str.contains('K'), col] = sale_k.astype(str)
    data.loc[data[col].str.contains('M'), col] = sale_m.astype(str)

data.NA_Sales.value_counts()

0       4367
0.02     536
0.03     523
0.01     521
0.05     519
        ... 
5.54       1
1.62       1
4.09       1
4.76       1
4.1        1
Name: NA_Sales, Length: 408, dtype: int64

In [166]:
to_M(data_1)

In [169]:
data_1.NA_Sales.value_counts()

0       4375
0.02     536
0.03     524
0.01     523
0.05     519
        ... 
5.54       1
1.62       1
4.09       1
4.76       1
4.1        1
Name: NA_Sales, Length: 408, dtype: int64

In [172]:
def to_int(data):
  col = data.columns[5:]
  data[col] = data[col].astype('float')
  data = data.reset_index().drop(columns='index')

to_int(data_1)

In [179]:
Total_sales = []
for i in data_1.index:
    Total_s = data_1.loc[i, 'NA_Sales':'Other_Sales'].sum()
    Total_sales.append(round(Total_s,2))

data_1['Total_Sales'] = Total_sales

In [180]:
data_1

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
0,Candace Kane's Candy Factory,DS,2008,Action,Destineer,0.04,0.00,0.00,0.00,0.04
1,The Munchables,Wii,2009,Action,Namco Bandai Games,0.17,0.00,0.00,0.01,0.18
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010,Adventure,Alchemist,0.00,0.00,0.02,0.00,0.02
3,Deal or No Deal: Special Edition,DS,2010,Misc,Zoo Games,0.04,0.00,0.00,0.00,0.04
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010,Platform,D3Publisher,0.12,0.09,0.00,0.04,0.25
...,...,...,...,...,...,...,...,...,...,...
16593,Ice Age 2: The Meltdown,GC,2006,Platform,Vivendi Games,0.15,0.04,0.00,0.01,0.20
16594,Rainbow Islands: Revolution,PSP,2005,Action,Rising Star Games,0.01,0.00,0.00,0.00,0.01
16595,NBA 2K16,PS3,2015,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13,0.79
16596,Toukiden: The Age of Demons,PSV,2013,Action,Tecmo Koei,0.05,0.05,0.25,0.03,0.38


In [181]:
data.to_csv("vgame_eda_1.csv", index=False)

# **2. Visuallization**

